# **Libraries Installation**

In [1]:
!pip install torch
!pip install pandas
!pip install transformers
!pip install bitsandbytes
!pip install peft
!pip install trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.4/296.4 kB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.1/280.1 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.7/105.7 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 42.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 55.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.8 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This be

# **Imports**

In [2]:
import torch, json
import pandas as pd
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model, PeftModel
from trl import SFTTrainer, DataCollatorForCompletionOnlyLM, SFTConfig
from torch.utils.data import Dataset

# Check device
device = "cuda" if torch.cuda.is_available() else "cpu"

# **HugginFace Login**

In [2]:
from huggingface_hub import notebook_login

# This will prompt you to enter your Hugging Face access token
notebook_login()

# **Tokenizer**

In [4]:
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3.1-8B")

# Assign a custom padding token to avoid using eos_token as padding
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


# **Quantization and LoRA Configs**

In [5]:
# Quantization Config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

# LoRA Config
lora_config = LoraConfig(
    r=16,  # LoRA Rank
    lora_alpha=20,  # LoRA Alpha
    target_modules="all-linear",
    lora_dropout=0.02,
    bias="none",
    task_type="CAUSAL_LM"
)

# Load and apply LoRA to the model
model = AutoModelForCausalLM.from_pretrained("meta-llama/Meta-Llama-3.1-8B")
model = get_peft_model(model, lora_config)

# Enable gradient computation for training
model.train()
model.enable_input_require_grads()
model.print_trainable_parameters()

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

trainable params: 41,943,040 || all params: 8,072,204,288 || trainable%: 0.5196


# **Data Cleaning and Preprocessing**

In [13]:
import pandas as pd

# Load the Parquet file
df_train = pd.read_parquet('/content/drive/MyDrive/ml-proj-nlp/data/train-00000-of-00001-8a9aa6c5fc4a1f00.parquet')
df_test = pd.read_parquet('/content/drive/MyDrive/ml-proj-nlp/data/test-00000-of-00001-5df4da5825ade20f.parquet')

# Convert to JSONL format
df_train.to_json('train.jsonl', orient='records', lines=True)
df_test.to_json('test.jsonl', orient='records', lines=True)

print("Files successfully converted to JSONL format.")

Files successfully converted to JSONL format.


In [17]:

df_train

,input,output
0,-- Database schema\n| department : Department_...,SELECT count(*) FROM head WHERE age > 56;
1,-- Database schema\n| department : Department_...,"SELECT name , born_state , age FROM head ORD..."
2,-- Database schema\n| department : Department_...,"SELECT creation , name , budget_in_billions ..."
3,-- Database schema\n| department : Department_...,"SELECT max(budget_in_billions) , min(budget_i..."
4,-- Database schema\n| department : Department_...,SELECT avg(num_employees) FROM department WHER...
...,...,...
16423,-- Database schema\n| film_text : film_id [ IN...,SELECT COUNT(T1.customer_id) FROM payment AS T...
16424,-- Database schema\n| film_text : film_id [ IN...,"SELECT T2.first_name, T2.last_name FROM paymen..."
16425,-- Database schema\n| film_text : film_id [ IN...,SELECT SUM(T5.amount) FROM address AS T1 INNER...
16426,-- Database schema\n| film_text : film_id [ IN...,SELECT COUNT(T1.amount) FROM payment AS T1 INN...


In [20]:
import os

def process_jsonl(input_file):
    new_lines = []
    with open(input_file, 'r') as f:
        for line in f:
            # load json
            data = json.loads(line)

            input_text = data['input']
            output_text = data['output']

            # split input text
            split_text = input_text.split("\n")
            if len(split_text) < 3:
                print(f'Error: {len(split_text)} splits in line: {line}')
                continue

            # Clean the schema
            schema = split_text[1]

            # split on the tables
            schema_table_split = schema.split("|")

            # format the schema
            schema = ""
            for i in range(1, len(schema_table_split) - 1):
                val = schema_table_split[i].strip()

                # split on the columns
                column_split = val.split(",")
                table_name = column_split[0].split(":")[0].strip()
                table_str = f"CREATE TABLE {table_name} ("
                val = val.replace(f"{table_name} :", "").strip()
                column_split = val.split(",")
                for j in range(len(column_split)):
                    col_name = column_split[j].split("[")[0].strip()
                    col_type = column_split[j].split("[")[1].split("]")[0].strip()

                    table_str += col_name + " " + col_type + ", "

                table_str = table_str[:-2] + ");\n"
                schema += table_str

            # Clean the query
            query = split_text[2]
            query = query.replace("-- -- ", "")
            query = query.replace(" ?", "?")
            query = query.strip()

            # update the data object
            data["query"] = query
            data["schema"] = schema
            del data["input"]

            # append to the new lines list
            line = json.dumps(data)
            new_lines.append(line)

    # write to a new file
    out_file = input_file + '.new'
    print(f'Writing {len(new_lines)} lines to file: {out_file}')
    with open(out_file, 'w') as f:
        for line in new_lines:
            f.write(line + '\n')

    # rename the new file to replace the original input file
    os.rename(out_file, input_file)
    print(f'Renamed {out_file} to {input_file}')


# Process both train and test JSONL files
process_jsonl('train.jsonl')
process_jsonl('test.jsonl')

print("Processing and renaming of JSONL files completed.")

Writing 16428 lines to file: train.jsonl.new
Renamed train.jsonl.new to train.jsonl
Writing 1034 lines to file: test.jsonl.new
Renamed test.jsonl.new to test.jsonl
Processing and renaming of JSONL files completed.


In [6]:
# Define the SQLDataset class for handling data
class SQLDataset(Dataset):
    def __init__(self, file_path, tokenizer, max_length=2048):
        self.tokenizer = tokenizer
        self.data = []
        self.max_length = max_length

        with open(file_path, 'r', encoding='utf-8') as f:
            for line in f:
                self.data.append(json.loads(line))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        record = self.data[idx]
        output = record["output"]
        query = record["query"]
        schema = record["schema"]

        input = f"Schema: {schema}\nInstructions: {query}\nAnswer: {output}"

        encoding = self.tokenizer(input, truncation=True, max_length=self.max_length, return_tensors="pt")

        return {'input_ids': encoding.input_ids.squeeze(0), 'attention_mask': encoding.attention_mask.squeeze(0)}

# Load training and evaluation data
file_path = "/content/drive/MyDrive/ml-proj-nlp/data/train.jsonl"
dataset = SQLDataset(file_path=file_path, tokenizer=tokenizer)

eval_file_path = "/content/drive/MyDrive/ml-proj-nlp/data/test.jsonl"
eval_dataset = SQLDataset(file_path=eval_file_path, tokenizer=tokenizer)

In [7]:
# Check the length of the dataset
print(f"Number of training samples: {len(dataset)}")
print(f"Number of evaluation samples: {len(eval_dataset)}")

num_examples_to_view = 1

# View training dataset examples
print("\nTraining Dataset Examples:")
for i in range(num_examples_to_view):
    sample = dataset[i]
    print(f"Example {i+1}:")
    print(f"Input IDs: {sample['input_ids']}")
    print(f"Attention Mask: {sample['attention_mask']}\n")

# View evaluation dataset examples
print("\nEvaluation Dataset Examples:")
for i in range(num_examples_to_view):
    sample = eval_dataset[i]
    print(f"Example {i+1}:")
    print(f"Input IDs: {sample['input_ids']}")
    print(f"Attention Mask: {sample['attention_mask']}\n")


Number of training samples: 16428
Number of evaluation samples: 1034

Training Dataset Examples:
Example 1:
Input IDs: tensor([128000,   8802,     25,  31876,  14700,   9476,    320,  27725,   3533,
          9403,     11,   4076,  16139,     11,  35386,  16139,     11,  65175,
          9403,     11,  28368,   1265,   1702,  91387,   9403,     11,  16610,
          2135,   7256,   5633,   9403,    317,  23421,  14700,   2010,    320,
          2025,   3533,   9403,     11,    836,  16139,     11,   9405,   4486,
         16139,     11,   4325,   9403,    317,  23421,  14700,   6373,    320,
         28414,   3533,   9403,     11,   2010,   3533,   9403,     11,  13643,
         30470,    287,  16139,    629,  56391,     25,   2650,   1690,  14971,
           315,    279,  26280,    527,   9191,   1109,    220,   3487,   5380,
         16533,     25,  19638,   1797,  29771,   4393,   2010,   5401,   4325,
           220,    871,    220,    220,   3487,     26])
Attention Mask: tensor([

# **Fine Tuning the Model**

In [8]:
# Data Collator (same as before)
instruction_template = "Schema:"
response_template = "Answer:"
collator = DataCollatorForCompletionOnlyLM(instruction_template=instruction_template, response_template=response_template, tokenizer=tokenizer, mlm=False)

# Training arguments
training_args = TrainingArguments(
    output_dir="./training",
    warmup_steps=5,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_checkpointing=True, # Enable gradient checkpointing
    gradient_accumulation_steps=64, # Increase gradient accumulation steps
    num_train_epochs=1,
    fp16=True, # Try enabling fp16
    optim="paged_lion_8bit",
    learning_rate=1e-4,
    weight_decay=0.01,
    save_strategy="epoch",
)

# Setup Trainer for the new model, passing max_seq_length and dataset_text_field directly
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    args=training_args,
    data_collator=collator,
    max_seq_length=2048,  # Set maximum sequence length here
    dataset_text_field="text"  # Field name for the text in your dataset
)

# Train the model
trainer.train()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': max_seq_length, dataset_text_field. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:283: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:321: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:488: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda'

Step,Training Loss


TrainOutput(global_step=256, training_loss=80.86465454101562, metrics={'train_runtime': 6773.6017, 'train_samples_per_second': 2.425, 'train_steps_per_second': 0.038, 'total_flos': 2.518848152239227e+17, 'train_loss': 80.86465454101562, 'epoch': 0.9973216459702946})

In [18]:
import torch

torch.cuda.empty_cache()  # Clears cache memory


# **Save Model**

In [9]:
# Save the model and tokenizer
model.save_pretrained('/content/drive/MyDrive/ml-proj-nlp/last/trained_model')
tokenizer.save_pretrained('/content/drive/MyDrive/ml-proj-nlp/last/trained_model')

('/content/drive/MyDrive/ml-proj-nlp/last/trained_model/tokenizer_config.json',
 '/content/drive/MyDrive/ml-proj-nlp/last/trained_model/special_tokens_map.json',
 '/content/drive/MyDrive/ml-proj-nlp/last/trained_model/tokenizer.json')

In [5]:
!pip install --upgrade transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 99.8 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.42.4
    Uninstalling transformers-4.42.4:
      Successfully uninstalled transformers-4.42.4


In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel
import torch

# Load the base model (assuming Meta-LLaMA-3.1 8B as an example)
base_model_name = "meta-llama/Meta-Llama-3.1-8B"
base_model = AutoModelForCausalLM.from_pretrained(base_model_name)

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model_name)

# Load the adapter model and apply it to the base model
adapter_model_path = '/content/drive/MyDrive/ml-proj-nlp/last/trained_model'
model = PeftModel.from_pretrained(base_model, adapter_model_path)

# Move the model to the appropriate device (GPU/CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Now the model with the adapter is loaded and ready for inference

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/826 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 4096)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaSdpaAttention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.02, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Li

In [ ]:
!pip install --upgrade transformers

# **Query Generation**

In [4]:
# Function to generate SQL query and handle post-processing to show only the Answer part
def generate_sql_query(schema, question, model, tokenizer):
    input_text = f"""Schema: {schema}\nInstructions: {question}\nAnswer:"""

    # Tokenize the input text and move the tensors to the appropriate device (CPU/GPU)
    input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to(device)
    attention_mask = tokenizer(input_text, return_tensors="pt").attention_mask.to(device)

    # Generate the SQL query using the model
    outputs = model.generate(
        input_ids,
        attention_mask=attention_mask,
        max_new_tokens=100,  # Limit the number of new tokens generated
        num_beams=5,
        early_stopping=True,
        pad_token_id=tokenizer.pad_token_id,
        eos_token_id=tokenizer.eos_token_id  # Ensure generation stops with EOS token
    )

    # Decode the output tokens into a string (the SQL query)
    decoded_output = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Extract the part of the string after "Answer:"
    answer_part = decoded_output.split("Answer:")[-1].strip()  # Get the content after "Answer:"

    # Post-process to keep only the first SQL query and ensure it ends with a semicolon
    sql_query = answer_part.split(";")[0].strip() + ";"

    return sql_query

# **Evaluation and Testing Results**

In [12]:
# Example usage for testing
schema = "CREATE TABLE head (age INTEGER);"
question = "How many heads of the departments are older than 56?"
sql_query = generate_sql_query(schema, question, model, tokenizer)
# sql_query = sql_query.split(";")[0] + ";"  # Keep only the first query before the first semicolon
print(sql_query)

Schema: CREATE TABLE head (age INTEGER);
Instructions: How many heads of the departments are older than 56?
Answer: SELECT COUNT(age) FROM head WHERE age > 56; SELECT COUNT(age) FROM head WHERE age > 56; SELECT COUNT(age) FROM head WHERE age > 56; SELECT COUNT(age) FROM head WHERE age > 56; SELECT COUNT(age) FROM head WHERE age > 56; SELECT COUNT(age) FROM head WHERE age > 56; SELECT COUNT(age) FROM head WHERE age


In [5]:
# Example usage for testing
# schema = "CREATE TABLE head (age INTEGER);"
# question = "How many heads of the departments are older than 56?"

schema = "CREATE TABLE head (age INTEGER);"
question = "How many heads of the departments are older than 40?"

# Generate and post-process the SQL query
sql_query = generate_sql_query(schema, question, model, tokenizer)

# Print the final post-processed SQL query
print(sql_query)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


SELECT COUNT(age) FROM head WHERE age > 40;


In [6]:
schema = "CREATE TABLE employee (name TEXT, age INTEGER, department TEXT);"
question = "Who is the oldest employee in the IT department?"

# Generate and post-process the SQL query
sql_query = generate_sql_query(schema, question, model, tokenizer)

# Print the final post-processed SQL query
print(sql_query)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


SELECT name FROM employee WHERE department = 'IT' ORDER BY age DESC LIMIT 1;


In [3]:
schema = "CREATE TABLE employee (name TEXT, age INTEGER, department TEXT);"
question = "What is the average age of employees in the marketing department?"

# Generate and post-process the SQL query
sql_query = generate_sql_query(schema, question, model, tokenizer)

# Print the final post-processed SQL query
print(sql_query)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


SELECT AVG(age) FROM employee WHERE department ='marketing';


In [4]:
schema = """
CREATE TABLE employee (
    employee_id INTEGER PRIMARY KEY,
    name TEXT,
    department_id INTEGER,
    salary INTEGER,
    age INTEGER,
    FOREIGN KEY (department_id) REFERENCES department(department_id)
);

CREATE TABLE department (
    department_id INTEGER PRIMARY KEY,
    department_name TEXT
);
"""
question = "Get all employees who work in the Sales department."


In [6]:
schema = """
CREATE TABLE employee (
    employee_id INTEGER PRIMARY KEY,
    name TEXT,
    department_id INTEGER,
    salary INTEGER,
    age INTEGER,
    FOREIGN KEY (department_id) REFERENCES department(department_id)
);

CREATE TABLE department (
    department_id INTEGER PRIMARY KEY,
    department_name TEXT
);
"""
question = "Find employees who are older than 40 and earn more than $5000 per month."


In [11]:
schema = """
CREATE TABLE author (
    author_id INTEGER PRIMARY KEY,
    author_name TEXT
);

CREATE TABLE book (
    book_id INTEGER PRIMARY KEY,
    book_title TEXT,
    publication_year INTEGER
);

CREATE TABLE author_book (
    author_id INTEGER,
    book_id INTEGER,
    FOREIGN KEY (author_id) REFERENCES author(author_id),
    FOREIGN KEY (book_id) REFERENCES book(book_id),
    PRIMARY KEY (author_id, book_id)
);
"""
question = "Find all books written by 'J.K. Rowling'."

In [7]:
# Generate and post-process the SQL query
sql_query = generate_sql_query(schema, question, model, tokenizer)

# Print the final post-processed SQL query
print(sql_query)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


SELECT name FROM employee WHERE age > 40 AND salary > 5000;


In [12]:
# Generate and post-process the SQL query
sql_query = generate_sql_query(schema, question, model, tokenizer)

# Print the final post-processed SQL query
print(sql_query)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


SELECT T1.book_title FROM book AS T1 INNER JOIN author_book AS T2 ON T1.book_id = T2.book_id INNER JOIN author AS T3 ON T2.author_id = T3.author_id WHERE T3.author_name = 'J.K. Rowling';


In [14]:
schema = """
CREATE TABLE author (
    author_id INTEGER PRIMARY KEY,
    author_name TEXT
);

CREATE TABLE book (
    book_id INTEGER PRIMARY KEY,
    book_title TEXT,
    publication_year INTEGER
);

CREATE TABLE author_book (
    author_id INTEGER,
    book_id INTEGER,
    FOREIGN KEY (author_id) REFERENCES author(author_id),
    FOREIGN KEY (book_id) REFERENCES book(book_id),
    PRIMARY KEY (author_id, book_id)
);
"""
question = "Find all books published after 2010."

# Generate and post-process the SQL query
sql_query = generate_sql_query(schema, question, model, tokenizer)

# Print the final post-processed SQL query
print(sql_query)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


SELECT book_title FROM book WHERE publication_year > 2010;


In [15]:
!pip install huggingface_hub

In [16]:
from huggingface_hub import HfApi

model.push_to_hub("text2sql_llama8b")  # Push your model to your HF account
tokenizer.push_to_hub("text2sql_llama8b")

adapter_model.safetensors:   0%|          | 0.00/168M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/source7th/text2sql_llama8b/commit/95fa5755fac0cbfae24792f49d89b98cbc01b38a', commit_message='Upload tokenizer', commit_description='', oid='95fa5755fac0cbfae24792f49d89b98cbc01b38a', pr_url=None, pr_revision=None, pr_num=None)

In [6]:
test_data = "/content/drive/MyDrive/ml-proj-nlp/data/test.jsonl"

In [4]:
import json

# Open and read the file
with open(test_data, 'r', encoding='utf-8') as f:
    # Loop through the first few lines
    num_lines_to_view = 5
    for i, line in enumerate(f):
        if i >= num_lines_to_view:
            break
        # Parse the JSONL line and print it
        record = json.loads(line)
        print(f"Record {i+1}: {record}\n")


Record 1: {'output': 'SELECT count(*) FROM singer;', 'query': 'How many singers do we have?', 'schema': 'CREATE TABLE stadium (Stadium_ID INT, Location TEXT, Name TEXT, Capacity INT, Highest INT, Lowest INT, Average INT);\nCREATE TABLE singer (Singer_ID INT, Name TEXT, Country TEXT, Song_Name TEXT, Song_release_year TEXT, Age INT, Is_male bool);\nCREATE TABLE concert (concert_ID INT, concert_Name TEXT, Theme TEXT, Stadium_ID TEXT, Year TEXT);\nCREATE TABLE singer_in_concert (concert_ID INT, Singer_ID TEXT);\n'}

Record 2: {'output': 'SELECT count(*) FROM singer;', 'query': 'What is the total number of singers?', 'schema': 'CREATE TABLE stadium (Stadium_ID INT, Location TEXT, Name TEXT, Capacity INT, Highest INT, Lowest INT, Average INT);\nCREATE TABLE singer (Singer_ID INT, Name TEXT, Country TEXT, Song_Name TEXT, Song_release_year TEXT, Age INT, Is_male bool);\nCREATE TABLE concert (concert_ID INT, concert_Name TEXT, Theme TEXT, Stadium_ID TEXT, Year TEXT);\nCREATE TABLE singer_in_con

In [23]:
import pandas as pd

# Load the JSONL file into a pandas DataFrame
df = pd.read_json(test_data, lines=True)

# Display the first few records
df

,output,query,schema
0,SELECT count(*) FROM singer;,How many singers do we have?,"CREATE TABLE stadium (Stadium_ID INT, Location..."
1,SELECT count(*) FROM singer;,What is the total number of singers?,"CREATE TABLE stadium (Stadium_ID INT, Location..."
2,"SELECT name , country , age FROM singer ORDE...","Show name, country, age for all singers ordere...","CREATE TABLE stadium (Stadium_ID INT, Location..."
3,"SELECT name , country , age FROM singer ORDE...","What are the names, countries, and ages for ev...","CREATE TABLE stadium (Stadium_ID INT, Location..."
4,"SELECT avg(age) , min(age) , max(age) FROM s...","What is the average, minimum, and maximum age ...","CREATE TABLE stadium (Stadium_ID INT, Location..."
...,...,...,...
1029,SELECT Citizenship FROM singer WHERE Birth_Yea...,What are the citizenships that are shared by s...,"CREATE TABLE singer (Singer_ID INT, Name TEXT,..."
1030,SELECT count(*) FROM Other_Available_Features;,How many available features are there in total?,CREATE TABLE Ref_Feature_Types (feature_type_c...
1031,SELECT T2.feature_type_name FROM Other_Availab...,What is the feature type name of feature AirCon?,CREATE TABLE Ref_Feature_Types (feature_type_c...
1032,SELECT T2.property_type_description FROM Prope...,Show the property type descriptions of propert...,CREATE TABLE Ref_Feature_Types (feature_type_c...


In [24]:
from transformers import logging

# Set logging level to ERROR to suppress info-level messages
logging.set_verbosity_error()

In [20]:
import re

# Function to normalize SQL queries
def normalize_sql(query):
    query = re.sub(r'\s+', ' ', query)
    query = query.replace('"', "'")
    query = query.replace("(", " ").replace(")", " ")
    query = query.lower().strip()

    return query

In [26]:
# Create a list to store the new data
new_data = []

# Iterate over the entire DataFrame
for index, row in df.iterrows():  # No need for sampling, iterate over all rows
    schema = row['schema']
    question = row['query']
    actual_output = row['output']

    # Generate the SQL query using the model
    generated_query = generate_sql_query(schema, question, model, tokenizer)

    # Normalize both queries
    normalized_actual = normalize_sql(actual_output)
    normalized_generated = normalize_sql(generated_query)

    # Compare the normalized queries
    is_match = normalized_actual == normalized_generated

    print(f"Is Match: {is_match}")

    # Append the result to new_data list
    new_data.append({
        'schema': schema,
        'query': question,
        'actual_output': actual_output,
        'generated_sql_query': generated_query,
        'is_match': is_match
    })

# Create a new DataFrame from the new_data list
new_df = pd.DataFrame(new_data)

Is Match: True
Is Match: True
Is Match: False
Is Match: False
Is Match: False
Is Match: False
Is Match: False
Is Match: False
Is Match: True
Is Match: True
Is Match: False
Is Match: False
Is Match: True
Is Match: True
Is Match: False
Is Match: False
Is Match: False
Is Match: False
Is Match: False
Is Match: False
Is Match: True
Is Match: True
Is Match: False
Is Match: False
Is Match: False
Is Match: False
Is Match: True
Is Match: True
Is Match: True
Is Match: True
Is Match: True
Is Match: False
Is Match: False
Is Match: False
Is Match: False
Is Match: False
Is Match: False
Is Match: False
Is Match: False
Is Match: False
Is Match: False
Is Match: False
Is Match: False
Is Match: True
Is Match: True
Is Match: True
Is Match: True
Is Match: False
Is Match: False
Is Match: False
Is Match: False
Is Match: False
Is Match: True
Is Match: False
Is Match: True
Is Match: True
Is Match: True
Is Match: False
Is Match: False
Is Match: False
Is Match: False
Is Match: False
Is Match: False
Is Match: Fal

In [28]:
# Save the new DataFrame to a CSV file
new_df.to_csv("/content/drive/MyDrive/ml-proj-nlp/generated_queries.csv", index=False)

In [27]:
new_df

,schema,query,actual_output,generated_sql_query,is_match
0,"CREATE TABLE stadium (Stadium_ID INT, Location...",How many singers do we have?,SELECT count(*) FROM singer;,SELECT count(*) FROM singer;,True
1,"CREATE TABLE stadium (Stadium_ID INT, Location...",What is the total number of singers?,SELECT count(*) FROM singer;,SELECT count(*) FROM singer;,True
2,"CREATE TABLE stadium (Stadium_ID INT, Location...","Show name, country, age for all singers ordere...","SELECT name , country , age FROM singer ORDE...","SELECT name, country, age FROM singer ORDER ...",False
3,"CREATE TABLE stadium (Stadium_ID INT, Location...","What are the names, countries, and ages for ev...","SELECT name , country , age FROM singer ORDE...","SELECT name, country, age FROM singer ORDER ...",False
4,"CREATE TABLE stadium (Stadium_ID INT, Location...","What is the average, minimum, and maximum age ...","SELECT avg(age) , min(age) , max(age) FROM s...","SELECT avg(Age), min(Age), max(Age) FROM sin...",False
...,...,...,...,...,...
1029,"CREATE TABLE singer (Singer_ID INT, Name TEXT,...",What are the citizenships that are shared by s...,SELECT Citizenship FROM singer WHERE Birth_Yea...,SELECT Citizenship FROM singer WHERE Birth_Yea...,True
1030,CREATE TABLE Ref_Feature_Types (feature_type_c...,How many available features are there in total?,SELECT count(*) FROM Other_Available_Features;,SELECT count(*) FROM Other_Available_Features;,True
1031,CREATE TABLE Ref_Feature_Types (feature_type_c...,What is the feature type name of feature AirCon?,SELECT T2.feature_type_name FROM Other_Availab...,SELECT T2.feature_type_name FROM Other_Availab...,True
1032,CREATE TABLE Ref_Feature_Types (feature_type_c...,Show the property type descriptions of propert...,SELECT T2.property_type_description FROM Prope...,SELECT property_type_description FROM Ref_Prop...,False


In [29]:
total_rows = len(new_df)
correct_matches = new_df['is_match'].sum()
accuracy_percentage = (correct_matches / total_rows) * 100

print(f"Exact Match Accuracy: {accuracy_percentage:.2f}%")


Exact Match Accuracy: 35.11%


In [ ]:
schema = """
CREATE TABLE author (
    author_id INTEGER PRIMARY KEY,
    author_name TEXT
);

CREATE TABLE book (
    book_id INTEGER PRIMARY KEY,
    book_title TEXT,
    publication_year INTEGER
);

CREATE TABLE author_book (
    author_id INTEGER,
    book_id INTEGER,
    FOREIGN KEY (author_id) REFERENCES author(author_id),
    FOREIGN KEY (book_id) REFERENCES book(book_id),
    PRIMARY KEY (author_id, book_id)
);
"""
question = "Find all books published after 2010."

# Generate and post-process the SQL query
sql_query = generate_sql_query(schema, question, model, tokenizer)

# Print the final post-processed SQL query
print(sql_query)

In [35]:
# List of schema and questions
data = [
    {
        "schema": """
        CREATE TABLE author (
            author_id INTEGER PRIMARY KEY,
            author_name TEXT
        );

        CREATE TABLE book (
            book_id INTEGER PRIMARY KEY,
            book_title TEXT,
            publication_year INTEGER
        );

        CREATE TABLE author_book (
            author_id INTEGER,
            book_id INTEGER,
            FOREIGN KEY (author_id) REFERENCES author(author_id),
            FOREIGN KEY (book_id) REFERENCES book(book_id),
            PRIMARY KEY (author_id, book_id)
        );
        """,
        "question": "Find all books published after 2010."
    },
    {
        "schema": """
        CREATE TABLE employee (
            employee_id INTEGER PRIMARY KEY,
            employee_name TEXT,
            department TEXT,
            salary INTEGER
        );

        CREATE TABLE department (
            department_id INTEGER PRIMARY KEY,
            department_name TEXT
        );
        """,
        "question": "What is the average salary in the IT department?"
    },
    {
        "schema": """
        CREATE TABLE sales (
            sale_id INTEGER PRIMARY KEY,
            product_name TEXT,
            sale_date DATE,
            quantity_sold INTEGER
        );
        """,
        "question": "List all products sold in 2021."
    },
    {
        "schema": """
        CREATE TABLE student (
            student_id INTEGER PRIMARY KEY,
            student_name TEXT,
            age INTEGER
        );

        CREATE TABLE enrollment (
            enrollment_id INTEGER PRIMARY KEY,
            student_id INTEGER,
            course_id INTEGER,
            semester TEXT,
            FOREIGN KEY (student_id) REFERENCES student(student_id)
        );

        CREATE TABLE course (
            course_id INTEGER PRIMARY KEY,
            course_name TEXT
        );
        """,
        "question": "Find the courses in which students over 20 years old are enrolled."
    },
    {
        "schema": """
        CREATE TABLE customer (
            customer_id INTEGER PRIMARY KEY,
            customer_name TEXT,
            city TEXT
        );

        CREATE TABLE order (
            order_id INTEGER PRIMARY KEY,
            order_date DATE,
            customer_id INTEGER,
            amount INTEGER,
            FOREIGN KEY (customer_id) REFERENCES customer(customer_id)
        );
        """,
        "question": "List the total number of orders made by customers from 'New York'."
    },
    {
        "schema": """
        CREATE TABLE hospital (
            hospital_id INTEGER PRIMARY KEY,
            hospital_name TEXT,
            location TEXT
        );

        CREATE TABLE patient (
            patient_id INTEGER PRIMARY KEY,
            patient_name TEXT,
            admitted_date DATE,
            hospital_id INTEGER,
            FOREIGN KEY (hospital_id) REFERENCES hospital(hospital_id)
        );
        """,
        "question": "How many patients were admitted to hospitals in 'Los Angeles' after 2020?"
    },
    {
        "schema": """
        CREATE TABLE teacher (
            teacher_id INTEGER PRIMARY KEY,
            teacher_name TEXT,
            subject TEXT
        );

        CREATE TABLE class (
            class_id INTEGER PRIMARY KEY,
            class_name TEXT,
            teacher_id INTEGER,
            FOREIGN KEY (teacher_id) REFERENCES teacher(teacher_id)
        );

        CREATE TABLE student (
            student_id INTEGER PRIMARY KEY,
            student_name TEXT,
            class_id INTEGER,
            FOREIGN KEY (class_id) REFERENCES class(class_id)
        );
        """,
        "question": "List all teachers who have more than 20 students."
    },
    {
        "schema": """
        CREATE TABLE restaurant (
            restaurant_id INTEGER PRIMARY KEY,
            restaurant_name TEXT,
            city TEXT
        );

        CREATE TABLE review (
            review_id INTEGER PRIMARY KEY,
            review_date DATE,
            restaurant_id INTEGER,
            rating INTEGER,
            FOREIGN KEY (restaurant_id) REFERENCES restaurant(restaurant_id)
        );
        """,
        "question": "Find the restaurants with an average rating above 4.5."
    },
    {
        "schema": """
        CREATE TABLE flight (
            flight_id INTEGER PRIMARY KEY,
            airline TEXT,
            departure_city TEXT,
            arrival_city TEXT,
            duration INTEGER
        );
        """,
        "question": "Which flights from 'New York' to 'Los Angeles' have a duration of less than 6 hours?"
    },
    {
        "schema": """
        CREATE TABLE match (
            match_id INTEGER PRIMARY KEY,
            team1 TEXT,
            team2 TEXT,
            score_team1 INTEGER,
            score_team2 INTEGER,
            match_date DATE
        );
        """,
        "question": "Which teams scored more than 3 goals in any match?"
    },
    {
        "schema": """
        CREATE TABLE movie (
            movie_id INTEGER PRIMARY KEY,
            title TEXT,
            release_year INTEGER,
            genre TEXT
        );

        CREATE TABLE actor (
            actor_id INTEGER PRIMARY KEY,
            actor_name TEXT
        );

        CREATE TABLE movie_actor (
            movie_id INTEGER,
            actor_id INTEGER,
            FOREIGN KEY (movie_id) REFERENCES movie(movie_id),
            FOREIGN KEY (actor_id) REFERENCES actor(actor_id)
        );
        """,
        "question": "List all actors who acted in movies released before 2000."
    },
    {
        "schema": """
        CREATE TABLE customer (
            customer_id INTEGER PRIMARY KEY,
            customer_name TEXT
        );

        CREATE TABLE invoice (
            invoice_id INTEGER PRIMARY KEY,
            customer_id INTEGER,
            amount INTEGER,
            date DATE,
            FOREIGN KEY (customer_id) REFERENCES customer(customer_id)
        );

        CREATE TABLE payment (
            payment_id INTEGER PRIMARY KEY,
            invoice_id INTEGER,
            payment_date DATE,
            FOREIGN KEY (invoice_id) REFERENCES invoice(invoice_id)
        );
        """,
        "question": "Find customers who haven't made any payments."
    },
    {
        "schema": """
        CREATE TABLE tournament (
            tournament_id INTEGER PRIMARY KEY,
            tournament_name TEXT
        );

        CREATE TABLE match (
            match_id INTEGER PRIMARY KEY,
            tournament_id INTEGER,
            team1 TEXT,
            team2 TEXT,
            winner TEXT,
            FOREIGN KEY (tournament_id) REFERENCES tournament(tournament_id)
        );
        """,
        "question": "List all tournaments where 'Team A' won more than 3 matches."
    },
    {
        "schema": """
        CREATE TABLE product (
            product_id INTEGER PRIMARY KEY,
            product_name TEXT,
            price INTEGER
        );

        CREATE TABLE order (
            order_id INTEGER PRIMARY KEY,
            product_id INTEGER,
            quantity INTEGER,
            FOREIGN KEY (product_id) REFERENCES product(product_id)
        );
        """,
        "question": "Find the total sales amount for each product."
    },
    {
        "schema": """
        CREATE TABLE bank_account (
            account_id INTEGER PRIMARY KEY,
            account_type TEXT,
            balance INTEGER
        );

        CREATE TABLE transaction (
            transaction_id INTEGER PRIMARY KEY,
            account_id INTEGER,
            transaction_date DATE,
            amount INTEGER,
            transaction_type TEXT,
            FOREIGN KEY (account_id) REFERENCES bank_account(account_id)
        );
        """,
        "question": "List all accounts with a balance below 1000 after transactions in 2021."
    },
    {
        "schema": """
        CREATE TABLE professor (
            professor_id INTEGER PRIMARY KEY,
            professor_name TEXT,
            department TEXT
        );

        CREATE TABLE research_project (
            project_id INTEGER PRIMARY KEY,
            project_title TEXT,
            department TEXT,
            budget INTEGER
        );
        """,
        "question": "Find the average budget of research projects in the 'Physics' department."
    },
    {
        "schema": """
        CREATE TABLE gym_member (
            member_id INTEGER PRIMARY KEY,
            member_name TEXT,
            membership_start DATE
        );

        CREATE TABLE gym_activity (
            activity_id INTEGER PRIMARY KEY,
            activity_name TEXT
        );

        CREATE TABLE participation (
            member_id INTEGER,
            activity_id INTEGER,
            participation_date DATE,
            FOREIGN KEY (member_id) REFERENCES gym_member(member_id),
            FOREIGN KEY (activity_id) REFERENCES gym_activity(activity_id)
        );
        """,
        "question": "List all gym members who participated in activities in 2021."
    },
    {
        "schema": """
        CREATE TABLE library (
            library_id INTEGER PRIMARY KEY,
            library_name TEXT
        );

        CREATE TABLE book (
            book_id INTEGER PRIMARY KEY,
            title TEXT,
            library_id INTEGER,
            FOREIGN KEY (library_id) REFERENCES library(library_id)
        );
        """,
        "question": "Find all libraries that have more than 1000 books."
    },
    {
        "schema": """
        CREATE TABLE employee (
            employee_id INTEGER PRIMARY KEY,
            name TEXT,
            department_id INTEGER,
            salary INTEGER,
            hire_date DATE
        );

        CREATE TABLE department (
            department_id INTEGER PRIMARY KEY,
            department_name TEXT
        );
        """,
        "question": "List employees who have been working in the 'HR' department for more than 5 years."
    },
    {
        "schema": """
        CREATE TABLE client (
            client_id INTEGER PRIMARY KEY,
            client_name TEXT
        );

        CREATE TABLE project (
            project_id INTEGER PRIMARY KEY,
            client_id INTEGER,
            project_name TEXT,
            start_date DATE,
            end_date DATE,
            FOREIGN KEY (client_id) REFERENCES client(client_id)
        );
        """,
        "question": "List all clients who have more than 3 ongoing projects."
    }
]

In [36]:
# Function to generate and store the results
results = []

# Iterate through each schema and question pair
for item in data:
    schema = item["schema"]
    question = item["question"]

    # Generate the SQL query using the model
    sql_query = generate_sql_query(schema, question, model, tokenizer)

    # Append the results
    results.append({
        'schema': schema,
        'question': question,
        'generated_sql_query': sql_query
    })

# Convert results into a DataFrame for better display
results_df = pd.DataFrame(results)

In [38]:
# Save the new DataFrame to a CSV file
results_df.to_csv("/content/drive/MyDrive/ml-proj-nlp/results.csv", index=False)

In [37]:
results_df

,schema,question,generated_sql_query
0,\n CREATE TABLE author (\n a...,Find all books published after 2010.,SELECT book_title FROM book WHERE publication_...
1,\n CREATE TABLE employee (\n ...,What is the average salary in the IT department?,SELECT AVG(T1.salary) FROM employee AS T1 INNE...
2,\n CREATE TABLE sales (\n sa...,List all products sold in 2021.,SELECT DISTINCT product_name FROM sales WHERE ...
3,\n CREATE TABLE student (\n ...,Find the courses in which students over 20 yea...,SELECT T1.course_name FROM course AS T1 INNER ...
4,\n CREATE TABLE customer (\n ...,List the total number of orders made by custom...,SELECT COUNT(T1.order_id) FROM order AS T1 INN...
5,\n CREATE TABLE hospital (\n ...,How many patients were admitted to hospitals i...,SELECT COUNT(T1.patient_id) FROM patient AS T1...
6,\n CREATE TABLE teacher (\n ...,List all teachers who have more than 20 students.,SELECT T1.teacher_name FROM teacher AS T1 INNE...
7,\n CREATE TABLE restaurant (\n ...,Find the restaurants with an average rating ab...,SELECT T1.restaurant_name FROM restaurant AS T...
8,\n CREATE TABLE flight (\n f...,Which flights from 'New York' to 'Los Angeles'...,SELECT flight_id FROM flight WHERE departure_c...
9,\n CREATE TABLE match (\n ma...,Which teams scored more than 3 goals in any ma...,SELECT team1 FROM match WHERE score_team1 > 3 ...


## **Load the model and use it**

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel
import torch

# Load the base model (assuming Meta-LLaMA-3.1 8B as an example)
base_model_name = "meta-llama/Meta-Llama-3.1-8B"
base_model = AutoModelForCausalLM.from_pretrained(base_model_name)

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model_name)

# Load the adapter model and apply it to the base model
adapter_model_path = '/content/drive/MyDrive/ml-proj-nlp/last/trained_model'
model = PeftModel.from_pretrained(base_model, adapter_model_path)

# Move the model to the appropriate device (GPU/CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Function to generate SQL query and handle post-processing to show only the Answer part
def generate_sql_query(schema, question, model, tokenizer):
    input_text = f"""Schema: {schema}\nInstructions: {question}\nAnswer:"""

    # Tokenize the input text and move the tensors to the appropriate device (CPU/GPU)
    input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to(device)
    attention_mask = tokenizer(input_text, return_tensors="pt").attention_mask.to(device)

    # Generate the SQL query using the model
    outputs = model.generate(
        input_ids,
        attention_mask=attention_mask,
        max_new_tokens=100,  # Limit the number of new tokens generated
        num_beams=5,
        early_stopping=True,
        pad_token_id=tokenizer.pad_token_id,
        eos_token_id=tokenizer.eos_token_id  # Ensure generation stops with EOS token
    )

    # Decode the output tokens into a string (the SQL query)
    decoded_output = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Extract the part of the string after "Answer:"
    answer_part = decoded_output.split("Answer:")[-1].strip()  # Get the content after "Answer:"

    # Post-process to keep only the first SQL query and ensure it ends with a semicolon
    sql_query = answer_part.split(";")[0].strip() + ";"

    return sql_query

In [ ]:
# Example usage

schema = "CREATE TABLE head (age INTEGER);"
question = "How many heads of the departments are older than 40?"

sql_query = generate_sql_query(schema, question, model, tokenizer)

print(sql_query)